In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

import numpy as np
import matplotlib.pyplot as plt
import os
from src import data
import json
from tqdm.auto import tqdm
from src.metrics import AggregateMetric
import logging

from src.utils import logging_utils
from src.models import load_model


# logging_utils.configure(level=logging.DEBUG)

In [3]:
# stats = {}

# dataset = data.load_dataset()
# for relation in dataset:
#     stats[relation.name] = {
#         "name": relation.name,
#         "category": relation.properties.relation_type,
#         "num_samples": len(relation.samples),
#         "|range|": len(set(relation.range))
#     }

# with open("stats/range_stats.json", "w") as f:
#     json.dump(stats, f, indent=2)

In [3]:
# model_name = "llama"

# mt = load_model(model_name, fp16=model_name != "gpt2-xl", device="cuda")

In [33]:
# tok_id = mt.tokenizer(" 1996").input_ids[2]
# mt.tokenizer.decode(tok_id)

'woman'

In [4]:
# " 1996".isnumeric()

In [5]:
# with open("stats/range_stats.json", "r") as f:
#     stats = json.load(f)

# dataset = data.load_dataset()
# prefix = " " if model_name != "llama" else ""

# for relation in dataset:
#     if relation.name not in stats:
#         continue
#     first_tokens = []
#     for obj in relation.range:
#         idx = 0
#         if model_name == "llama":
#             idx = 2 if obj.isnumeric() else 1
#         tok_id = mt.tokenizer(prefix + obj).input_ids[idx]
#         # print(idx, tok_id, f"`{obj}` | `{mt.tokenizer.decode(tok_id)}`")
#         first_tokens.append(mt.tokenizer.decode(tok_id))
#     stats[relation.name][model_name] = len(set(first_tokens))

# with open("stats/range_stats.json", "w") as f:
#     json.dump(stats, f, indent=2)

## Table 10

In [6]:
with open("stats/range_stats.json", "r") as f:
    stats = json.load(f)

In [29]:
relation_stats = [
    stats[relation] 
    for relation in stats 
]

relation_stats = sorted(
    relation_stats, key=lambda x: x["name"],
)

table = []
unambiguous_range = 0
gptj_percent = []
gpt2xl_percent = []
llama_percent = []

def check_range_mismatch(relation_stat):
    for unique_first_tokens in [relation_stat["gptj"], relation_stat["gpt2-xl"], relation_stat["llama"]]:
        if unique_first_tokens != relation_stat["|range|"]:
            return False
    return True

for relation in relation_stats:
    if not check_range_mismatch(relation):
        gptj = relation["gptj"] * 100 / relation["|range|"]
        gpt2xl = relation["gpt2-xl"] * 100 / relation["|range|"]
        llama = relation["llama"] * 100 / relation["|range|"]
        table.append({
            "Relation": f'{relation["name"]}',
            "|range|": f'${relation["|range|"]}$',
            "GPT-J": f'${gptj:.1f}\%\;({relation["gptj"]})$',
            "GPT2-xl": f'${gpt2xl:.1f}\%\;({relation["gpt2-xl"]})$',
            "LLaMa-13B": f'${llama:.1f}\%\;({relation["llama"]})$',
        })
        gptj_percent.append(gptj)
        gpt2xl_percent.append(gpt2xl)
        if relation["name"].startswith("president"):
            continue
        llama_percent.append(llama)
    else:
        unambiguous_range += relation["|range|"]
        gptj_percent.append(100)
        gpt2xl_percent.append(100)
        llama_percent.append(100)

import pandas as pd
df = pd.DataFrame(table)
print(df.style.hide(axis = "index").to_latex())

\begin{tabular}{lllll}
Relation & |range| & GPT-J & GPT2-xl & LLaMa-13B \\
adjective antonym & $95$ & $100.0\%\;(95)$ & $100.0\%\;(95)$ & $98.9\%\;(94)$ \\
adjective comparative & $57$ & $100.0\%\;(57)$ & $100.0\%\;(57)$ & $93.0\%\;(53)$ \\
adjective superlative & $79$ & $97.5\%\;(77)$ & $97.5\%\;(77)$ & $98.7\%\;(78)$ \\
city in country & $21$ & $95.2\%\;(20)$ & $95.2\%\;(20)$ & $95.2\%\;(20)$ \\
company CEO & $287$ & $72.5\%\;(208)$ & $72.5\%\;(208)$ & $67.6\%\;(194)$ \\
company hq & $163$ & $100.0\%\;(163)$ & $100.0\%\;(163)$ & $93.3\%\;(152)$ \\
country currency & $23$ & $100.0\%\;(23)$ & $100.0\%\;(23)$ & $91.3\%\;(21)$ \\
landmark in country & $91$ & $100.0\%\;(91)$ & $100.0\%\;(91)$ & $97.8\%\;(89)$ \\
person father & $968$ & $41.3\%\;(400)$ & $41.3\%\;(400)$ & $38.9\%\;(377)$ \\
person lead singer of band & $21$ & $85.7\%\;(18)$ & $85.7\%\;(18)$ & $85.7\%\;(18)$ \\
person mother & $962$ & $39.5\%\;(380)$ & $39.5\%\;(380)$ & $31.9\%\;(307)$ \\
person occupation & $31$ & $100.0\%

In [30]:
len(relation_stats) - len(table)

26

In [31]:
for avg in [gptj_percent, gpt2xl_percent, llama_percent]:
    print(f"${np.mean(avg):.2f} \pm {np.std(avg):.2f}$")

$93.17 \pm 15.04$
$93.17 \pm 15.04$
$92.17 \pm 15.57$
